# engine

In [100]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [101]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2022-06-15 11:32:29,093 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,094 INFO sqlalchemy.engine.Engine select 'hello world'
2022-06-15 11:32:29,095 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()
[('hello world',)]
2022-06-15 11:32:29,097 INFO sqlalchemy.engine.Engine ROLLBACK


In [102]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()

2022-06-15 11:32:29,158 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,160 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-06-15 11:32:29,160 INFO sqlalchemy.engine.Engine [generated in 0.00206s] ()
2022-06-15 11:32:29,162 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-06-15 11:32:29,163 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ((1, 1), (2, 4))
2022-06-15 11:32:29,164 INFO sqlalchemy.engine.Engine COMMIT


In [103]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2022-06-15 11:32:29,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,233 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-06-15 11:32:29,234 INFO sqlalchemy.engine.Engine [cached since 0.07173s ago] ((6, 8), (9, 10))
2022-06-15 11:32:29,235 INFO sqlalchemy.engine.Engine COMMIT


In [104]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2022-06-15 11:32:29,291 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,292 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-06-15 11:32:29,293 INFO sqlalchemy.engine.Engine [generated in 0.00272s] ()
x: 1, y: 1
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2022-06-15 11:32:29,296 INFO sqlalchemy.engine.Engine ROLLBACK


In [105]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y >= :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2022-06-15 11:32:29,342 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,344 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y >= ?
2022-06-15 11:32:29,345 INFO sqlalchemy.engine.Engine [generated in 0.00295s] (2,)
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2022-06-15 11:32:29,347 INFO sqlalchemy.engine.Engine ROLLBACK


In [106]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()

2022-06-15 11:32:29,399 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,400 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-06-15 11:32:29,401 INFO sqlalchemy.engine.Engine [cached since 0.2391s ago] ((11, 12), (13, 14))
2022-06-15 11:32:29,403 INFO sqlalchemy.engine.Engine COMMIT


In [107]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2022-06-15 11:32:29,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,456 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-06-15 11:32:29,457 INFO sqlalchemy.engine.Engine [generated in 0.00291s] (6,)
x: 6, y: 8
x: 9, y: 10
x: 11, y: 12
x: 13, y: 14
2022-06-15 11:32:29,460 INFO sqlalchemy.engine.Engine ROLLBACK


In [108]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2022-06-15 11:32:29,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,510 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-06-15 11:32:29,511 INFO sqlalchemy.engine.Engine [cached since 0.05665s ago] (6,)
x: 6, y: 8
x: 9, y: 10
x: 11, y: 12
x: 13, y: 14
2022-06-15 11:32:29,513 INFO sqlalchemy.engine.Engine ROLLBACK


In [109]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}]
    )
    session.commit()

2022-06-15 11:32:29,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:29,578 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-06-15 11:32:29,579 INFO sqlalchemy.engine.Engine [generated in 0.00098s] ((11, 9), (15, 13))
2022-06-15 11:32:29,580 INFO sqlalchemy.engine.Engine COMMIT


In [110]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [111]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [112]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [113]:
user_table.c.keys()

['id', 'name', 'fullname']

In [114]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [115]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [116]:
metadata_obj.create_all(engine)

2022-06-15 11:32:30,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:30,022 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-06-15 11:32:30,023 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,024 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-06-15 11:32:30,025 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,026 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-06-15 11:32:30,027 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,029 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-06-15 11:32:30,029 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,031 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-06-15 11:32:30,032 INFO sqlalchemy.engine.Engine [no key 0.00121s] ()
2022-06-15 11:32:30,034 INFO sqlalchemy.engine.Engine 
C

In [117]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [118]:
mapper_registry.metadata

MetaData()

In [119]:
Base = mapper_registry.generate_base()

In [120]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = "user_account"

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"User(id={self.id!r}, email_address={self.email_address!r})"

In [121]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [122]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [123]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

In [124]:
mapper_registry.metadata.create_all(engine)

2022-06-15 11:32:30,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:30,513 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-06-15 11:32:30,514 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,515 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-06-15 11:32:30,516 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,517 INFO sqlalchemy.engine.Engine COMMIT


In [125]:
Base.metadata.create_all(engine)

2022-06-15 11:32:30,572 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:30,574 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-06-15 11:32:30,575 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,576 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-06-15 11:32:30,577 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,579 INFO sqlalchemy.engine.Engine COMMIT


In [126]:
mapper_registry = registry()
Base = mapper_registry.generate_base()

class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

In [127]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2022-06-15 11:32:30,678 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:30,680 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-06-15 11:32:30,681 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,683 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-06-15 11:32:30,684 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-06-15 11:32:30,685 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-06-15 11:32:30,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,688 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-06-15 11:32:30,688 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-15 11:32:30,689 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [128]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

In [129]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [130]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [131]:
compiled = stmt.compile()

In [132]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [133]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-06-15 11:32:31,141 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:31,144 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-06-15 11:32:31,144 INFO sqlalchemy.engine.Engine [generated in 0.00299s] ('spongebob', 'Spongebob Squarepants')
2022-06-15 11:32:31,146 INFO sqlalchemy.engine.Engine COMMIT


In [134]:
result.inserted_primary_key

(1,)

In [135]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ]
    )
    conn.commit()

2022-06-15 11:32:31,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:31,241 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-06-15 11:32:31,241 INFO sqlalchemy.engine.Engine [generated in 0.00238s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-06-15 11:32:31,242 INFO sqlalchemy.engine.Engine COMMIT


In [136]:
from sqlalchemy import bindparam, select
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id = scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-06-15 11:32:31,326 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:31,328 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-06-15 11:32:31,329 INFO sqlalchemy.engine.Engine [generated in 0.00286s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-06-15 11:32:31,330 INFO sqlalchemy.engine.Engine COMMIT


In [137]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt,
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [138]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [139]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt,
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


In [140]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [141]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-06-15 11:32:31,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:31,642 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 11:32:31,643 INFO sqlalchemy.engine.Engine [generated in 0.00233s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-06-15 11:32:31,645 INFO sqlalchemy.engine.Engine ROLLBACK


In [142]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-06-15 11:32:31,701 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:31,706 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 11:32:31,707 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ('spongebob',)
(User('spongebob', 'Spongebob Squarepants'),)
2022-06-15 11:32:31,709 INFO sqlalchemy.engine.Engine ROLLBACK


In [143]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [144]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [145]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [146]:
row = session.execute(select(User)).first()

2022-06-15 11:32:32,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:32,563 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-06-15 11:32:32,564 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()


In [147]:
row

(User('spongebob', 'Spongebob Squarepants'),)

In [148]:
row[0]

User('spongebob', 'Spongebob Squarepants')

In [149]:
user = session.scalars(select(User)).first()

2022-06-15 11:32:32,734 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-06-15 11:32:32,735 INFO sqlalchemy.engine.Engine [cached since 0.1727s ago] ()


In [150]:
user

User('spongebob', 'Spongebob Squarepants')

In [151]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [152]:
row = session.execute(select(User.name, User.fullname)).first()

2022-06-15 11:32:32,888 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-06-15 11:32:32,890 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ()


In [153]:
row

('spongebob', 'Spongebob Squarepants')

In [154]:
session.execute(
    select(User.name, Address).
    where(User.id == Address.user_id).
    order_by(Address.id)
).all()

2022-06-15 11:32:32,998 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.user_id, address.email_address 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2022-06-15 11:32:33,000 INFO sqlalchemy.engine.Engine [generated in 0.00228s] ()


[('spongebob', Address('spongebob@sqlalchemy.org')),
 ('sandy', Address('sandy@sqlalchemy.org')),
 ('sandy', Address('sandy@squirrelpower.org'))]

In [155]:
from sqlalchemy import func, cast

stmt = (
    select(("Username: " + user_table.c.name).label("username")).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2022-06-15 11:32:33,060 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:33,061 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-06-15 11:32:33,063 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-06-15 11:32:33,064 INFO sqlalchemy.engine.Engine ROLLBACK


In [156]:
stmt = (
    select(
        text("'some phrases'"), user_table.c.name,
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-06-15 11:32:33,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:33,125 INFO sqlalchemy.engine.Engine SELECT 'some phrases', user_account.name 
FROM user_account ORDER BY user_account.name
2022-06-15 11:32:33,126 INFO sqlalchemy.engine.Engine [generated in 0.00303s] ()
[('some phrases', 'patrick'), ('some phrases', 'sandy'), ('some phrases', 'spongebob')]
2022-06-15 11:32:33,128 INFO sqlalchemy.engine.Engine ROLLBACK


In [157]:
from sqlalchemy import literal_column
stmt = (
    select(
        literal_column("'some phrase'").label("p"), user_table.c.name,
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2022-06-15 11:32:33,312 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:33,314 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2022-06-15 11:32:33,315 INFO sqlalchemy.engine.Engine [generated in 0.00282s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2022-06-15 11:32:33,317 INFO sqlalchemy.engine.Engine ROLLBACK


In [158]:
print(user_table.c.name == "squidward")

user_account.name = :name_1


In [159]:
print(address_table.c.user_id > 10)

address.user_id > :user_id_1


In [160]:
print(select(user_table).where(user_table.c.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [161]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == "squidward").
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [162]:
print(
    select(address_table.c.email_address).
    where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [163]:
from sqlalchemy import and_, or_

In [164]:
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


In [165]:
print(
    select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


In [166]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


In [167]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


In [168]:
print(select(user_table.c.name, address_table.c.email_address).join_from(user_table, Address))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [169]:
print(select(user_table.c.name, address_table.c.email_address).join(address_table))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [170]:
print(select(address_table.c.email_address).select_from(user_table).join(address_table))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [171]:
from sqlalchemy import func
print(
    select(func.count("*")).select_from(user_table)
)

SELECT count(:count_2) AS count_1 
FROM user_account


In [172]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == Address.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [173]:
print(select(user_table).join(address_table, isouter=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [174]:
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


In [175]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


In [176]:
print(select(user_table).order_by(user_table.c.name.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name DESC


In [177]:
count_fn = func.count(user_table.c.id)
print(count_fn)

count(user_account.id)


In [178]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count")).
        join(Address).
        group_by(User.name).
        having(func.count(Address.id) > 1)
    )
    print(result.all())

2022-06-15 11:32:34,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:34,633 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2022-06-15 11:32:34,634 INFO sqlalchemy.engine.Engine [generated in 0.00248s] (1,)
[('sandy', 2)]
2022-06-15 11:32:34,636 INFO sqlalchemy.engine.Engine ROLLBACK


In [179]:
from sqlalchemy import func, desc
stmt = (
    select(
        Address.user_id,
        func.count(Address.id).label("num_address"),
    ).
    group_by("user_id").order_by("user_id", desc("num_address"))
)
print(stmt)

SELECT address.user_id, count(address.id) AS num_address 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_address DESC


In [180]:
user_alias1 = user_table.alias()
user_alias2 = user_table.alias()
print(
    select(user_alias1.c.name, user_alias2.c.name).
    join_from(user_alias1, user_alias2, user_alias1.c.id > user_alias2.c.id)
)

SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id > user_account_2.id


In [181]:
from sqlalchemy.orm import aliased

address_alias1 = aliased(Address)
address_alias2 = aliased(Address)

print(
    select(User).
    join_from(User, address_alias1).
    where(address_alias1.email_address == "patrick@aol.com").
    join_from(User, address_alias2).
    where(address_alias2.email_address == "patrick@gmail.com")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


In [182]:
type(Address)

sqlalchemy.orm.decl_api.DeclarativeMeta

In [183]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).subquery()

In [184]:
print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


In [185]:
print(select(subq.c.count, subq.c.user_id))

SELECT anon_1.count, anon_1.user_id 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


In [186]:
stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)
print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


In [187]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id,
).group_by(address_table.c.user_id).cte()
print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


In [188]:
stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)
print(stmt)

WITH anon_1 AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id


In [189]:
subq = select(Address).where(~Address.email_address.like("%@aol.com")).subquery()
address_subq = aliased(Address, subq)
stmt = select(User, address_subq).join_from(User, address_subq).order_by(User.id, address_subq.id)

In [190]:
cte_obj = select(Address).where(~Address.email_address.like("%@aol.com")).cte()
address_cte = aliased(Address, cte_obj)
stmt = select(User, address_cte).join_from(User, address_cte).order_by(User.id, address_cte.id)
with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user} {address}")

2022-06-15 11:32:35,269 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,274 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT address.id AS id, address.user_id AS user_id, address.email_address AS email_address 
FROM address 
WHERE address.email_address NOT LIKE ?)
 SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.user_id, anon_1.email_address 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2022-06-15 11:32:35,276 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ('%@aol.com',)
User('spongebob', 'Spongebob Squarepants') Address('spongebob@sqlalchemy.org')
User('sandy', 'Sandy Cheeks') Address('sandy@sqlalchemy.org')
User('sandy', 'Sandy Cheeks') Address('sandy@squirrelpower.org')
2022-06-15 11:32:35,278 INFO sqlalchemy.engine.Engine ROLLBACK


In [191]:
subq = select(func.count(address_table.c.id)).where(user_table.c.id == address_table.c.user_id).scalar_subquery()
print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


In [192]:
print(subq == 5)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


In [193]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


In [194]:
stmt = select(
    user_table.c.name,
    address_table.c.email_address,
    subq.label("address_count")
).join_from(user_table, address_table).order_by(user_table.c.id, address_table.c.user_id)
#print(stmt)

In [195]:
subq = select(func.count(address_table.c.id)).where(user_table.c.id == address_table.c.user_id).scalar_subquery().correlate(user_table)

In [196]:
with engine.connect() as conn:
    result = conn.execute(
        select(
            user_table.c.name,
            address_table.c.email_address,
            subq.label("address_count"),
        ).
        join_from(user_table, address_table).
        order_by(user_table.c.id, address_table.c.id)
    )
    print(result.all())

2022-06-15 11:32:35,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,587 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2022-06-15 11:32:35,588 INFO sqlalchemy.engine.Engine [generated in 0.00327s] ()
[('spongebob', 'spongebob@sqlalchemy.org', 1), ('sandy', 'sandy@sqlalchemy.org', 2), ('sandy', 'sandy@squirrelpower.org', 2)]
2022-06-15 11:32:35,590 INFO sqlalchemy.engine.Engine ROLLBACK


In [197]:
from sqlalchemy import union_all

In [198]:
stmt1 = select(user_table).where(user_table.c.name == "sandy")
stmt2 = select(user_table).where(user_table.c.name == "spongebob")
u = union_all(stmt1, stmt2)
with engine.connect() as conn:
    result = conn.execute(u)
    print(result)

2022-06-15 11:32:35,676 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,678 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 11:32:35,679 INFO sqlalchemy.engine.Engine [generated in 0.00255s] ('sandy', 'spongebob')
2022-06-15 11:32:35,681 INFO sqlalchemy.engine.Engine ROLLBACK


In [199]:
u_subq = u.subquery()
stmt = (
    select(u_subq.c.name, address_table.c.email_address).
    join_from(address_table, u_subq).
    order_by(u_subq.c.name, address_table.c.email_address)
)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-06-15 11:32:35,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,751 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2022-06-15 11:32:35,752 INFO sqlalchemy.engine.Engine [generated in 0.00395s] ('sandy', 'spongebob')
[('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'), ('spongebob', 'spongebob@sqlalchemy.org')]
2022-06-15 11:32:35,754 INFO sqlalchemy.engine.Engine ROLLBACK


In [200]:
stmt1 = select(User).where(User.name == "sandy")
stmt2 = select(User).where(User.name == "spongebob")
u = union_all(stmt1, stmt2)

In [201]:
orm_stmt = select(User).from_statement(u)
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-06-15 11:32:35,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,857 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 11:32:35,858 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ('sandy', 'spongebob')
User('sandy', 'Sandy Cheeks')
User('spongebob', 'Spongebob Squarepants')
2022-06-15 11:32:35,861 INFO sqlalchemy.engine.Engine ROLLBACK


In [202]:
user_alias = aliased(User, u.subquery())
orm_stmt = select(user_alias).order_by(user_alias.id)
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-06-15 11:32:35,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,911 INFO sqlalchemy.engine.Engine SELECT anon_1.id, anon_1.name, anon_1.fullname 
FROM (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ORDER BY anon_1.id
2022-06-15 11:32:35,912 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('sandy', 'spongebob')
User('spongebob', 'Spongebob Squarepants')
User('sandy', 'Sandy Cheeks')
2022-06-15 11:32:35,915 INFO sqlalchemy.engine.Engine ROLLBACK


In [203]:
subq = (
    select(func.count(address_table.c.id)).
    where(user_table.c.id == address_table.c.user_id).
    group_by(address_table.c.user_id).
    having(func.count(address_table.c.id) > 1)
).exists()

with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name).where(subq)
    )
    print(result.all())

2022-06-15 11:32:35,968 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:35,970 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE EXISTS (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id GROUP BY address.user_id 
HAVING count(address.id) > ?)
2022-06-15 11:32:35,971 INFO sqlalchemy.engine.Engine [generated in 0.00316s] (1,)
[('sandy',)]
2022-06-15 11:32:35,973 INFO sqlalchemy.engine.Engine ROLLBACK


In [204]:
subq = select(address_table.c.id).where(user_table.c.id == address_table.c.user_id).exists()
with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name).where(~subq))
    print(result.all())

2022-06-15 11:32:36,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:36,023 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE NOT (EXISTS (SELECT address.id 
FROM address 
WHERE user_account.id = address.user_id))
2022-06-15 11:32:36,024 INFO sqlalchemy.engine.Engine [generated in 0.00222s] ()
[('patrick',)]
2022-06-15 11:32:36,025 INFO sqlalchemy.engine.Engine ROLLBACK


In [205]:
print(select(func.count()).select_from(user_table))

SELECT count(*) AS count_1 
FROM user_account


In [206]:
print(select(func.lower("A String With Much UPPERCASE")))

SELECT lower(:lower_2) AS lower_1


In [207]:
stmt = select(func.now())
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-06-15 11:32:36,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:36,164 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now_1
2022-06-15 11:32:36,165 INFO sqlalchemy.engine.Engine [generated in 0.00333s] ()
[(datetime.datetime(2022, 6, 15, 11, 32, 36),)]
2022-06-15 11:32:36,166 INFO sqlalchemy.engine.Engine ROLLBACK


In [208]:
print(select(func.some_crazy_function(user_table.c.name, 17)))

SELECT some_crazy_function(user_account.name, :some_crazy_function_2) AS some_crazy_function_1 
FROM user_account


In [209]:
from sqlalchemy.dialects import postgresql
print(select(func.now()).compile(dialect=postgresql.dialect()))

SELECT now() AS now_1


In [210]:
from sqlalchemy.dialects import oracle
print(select(func.now()).compile(dialect=oracle.dialect()))

SELECT CURRENT_TIMESTAMP AS now_1 FROM DUAL


In [211]:
func.now().type

DateTime()

In [212]:
from sqlalchemy import JSON
function_expr = func.json_object("{a, 1, b, 'def', c, 3.5}", type_=JSON)

In [213]:
stmt = select(function_expr["def"])
print(stmt)

SELECT json_object(:json_object_1)[:json_object_2] AS anon_1


In [214]:
m1 = func.max(Column("some_int", Integer))
m1.type

Integer()

In [215]:
m2 = func.max(Column("some_str", String))
m2.type

String()

In [216]:
func.now().type

DateTime()

In [217]:
func.current_date().type

Date()

In [218]:
func.concat("x", "y").type

String()

In [219]:
func.upper("lowercase").type

NullType()

In [220]:
print(select(func.upper("lowercase") + "_suffix"))

SELECT upper(:upper_1) || :upper_2 AS anon_1


In [221]:
func.count().type

Integer()

In [222]:
func.json_object('{"a", "b"}').type

NullType()

In [223]:
stmt = select(
    func.row_number().over(partition_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address,
).select_from(user_table).join(address_table)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-06-15 11:32:37,117 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:37,119 INFO sqlalchemy.engine.Engine SELECT row_number() OVER (PARTITION BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-06-15 11:32:37,120 INFO sqlalchemy.engine.Engine [generated in 0.00283s] ()
[(1, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (1, 'spongebob', 'spongebob@sqlalchemy.org')]
2022-06-15 11:32:37,122 INFO sqlalchemy.engine.Engine ROLLBACK


In [224]:
stmt = select(
    func.count().over(order_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address,
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())


2022-06-15 11:32:37,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:37,174 INFO sqlalchemy.engine.Engine SELECT count(*) OVER (ORDER BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-06-15 11:32:37,175 INFO sqlalchemy.engine.Engine [generated in 0.00262s] ()
[(2, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (3, 'spongebob', 'spongebob@sqlalchemy.org')]
2022-06-15 11:32:37,177 INFO sqlalchemy.engine.Engine ROLLBACK


In [225]:
print(
    func.unnest(
        func.percentile_disc([0.25, 0.5, 0.75, 1]).within_group(user_table.c.name)
    )
)

unnest(percentile_disc(:percentile_disc_1) WITHIN GROUP (ORDER BY user_account.name))


In [226]:
stmt = select(
    func.count(address_table.c.email_address).filter(user_table.c.name == "sandy"),
    func.count(address_table.c.email_address).filter(user_table.c.name == "spongebob"),
).select_from(user_table).join(address_table)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-06-15 11:32:37,265 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:37,267 INFO sqlalchemy.engine.Engine SELECT count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_1, count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_2 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-06-15 11:32:37,268 INFO sqlalchemy.engine.Engine [generated in 0.00267s] ('sandy', 'spongebob')
[(2, 1)]
2022-06-15 11:32:37,270 INFO sqlalchemy.engine.Engine ROLLBACK


In [227]:
onetwothree = func.json_each('["one", "two", "three"]').table_valued("value")
stmt = select(onetwothree).where(onetwothree.c.value.in_(["two", "three"]))
with engine.connect() as conn:  
    result = conn.execute(stmt)
    print(result.all())

2022-06-15 11:32:37,320 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:37,322 INFO sqlalchemy.engine.Engine SELECT anon_1.value 
FROM json_each(?) AS anon_1 
WHERE anon_1.value IN (?, ?)
2022-06-15 11:32:37,323 INFO sqlalchemy.engine.Engine [generated in 0.00310s] ('["one", "two", "three"]', 'two', 'three')
[('two',), ('three',)]
2022-06-15 11:32:37,326 INFO sqlalchemy.engine.Engine ROLLBACK


In [228]:
from sqlalchemy import select, func
stmt = select(func.json_array_elements('["one", "two"]').column_valued("x"))
print(stmt)

SELECT x 
FROM json_array_elements(:json_array_elements_1) AS x


In [229]:
from sqlalchemy.dialects import oracle
stmt = select(func.scalar_strings(5).column_valued("s"))
print(stmt.compile(dialect=oracle.dialect()))

SELECT COLUMN_VALUE s 
FROM TABLE (scalar_strings(:scalar_strings_1)) s


In [230]:
from sqlalchemy import cast
stmt = select(cast(user_table.c.id, String))
with engine.connect() as conn:
    result = conn.execute(stmt)
    result.all()

2022-06-15 11:32:37,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:32:37,528 INFO sqlalchemy.engine.Engine SELECT CAST(user_account.id AS VARCHAR) AS id 
FROM user_account
2022-06-15 11:32:37,529 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ()
2022-06-15 11:32:37,531 INFO sqlalchemy.engine.Engine ROLLBACK


In [232]:
from sqlalchemy import JSON
print(cast("{'a': 1, 'b': 2}", JSON)["a"])

CAST(:param_1 AS JSON)[:param_2]


In [234]:
import json
from sqlalchemy import JSON
from sqlalchemy import type_coerce
from sqlalchemy.dialects import mysql

s = select(
    type_coerce(
        {'some_key': {'foo': 'bar'}}, JSON
    )['some_key']
)
print(s.compile(dialect=mysql.dialect()))

SELECT JSON_EXTRACT(%s, %s) AS anon_1


In [235]:
from sqlalchemy import update
stmt = (
    update(user_table).where(user_table.c.name == "patrick").
    values(fullname="Patrick the Star")
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


In [236]:
stmt = (
    update(user_table).
    values(fullname="Username: " + user_table.c.name)
)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


In [237]:
from sqlalchemy import bindparam
stmt = (
    update(user_table).
    where(user_table.c.name == bindparam("oldname")).
    values(name=bindparam("newname"))
)
with engine.connect() as conn:
    conn.execute(
        stmt,
        [
            {"oldname": "jack", "newname": "ed"},
            {"oldname": "wendy", "newname": "mary"},
            {"oldname": "jim", "newname": "jake"},
        ]
    )

2022-06-15 11:41:58,040 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:41:58,042 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2022-06-15 11:41:58,044 INFO sqlalchemy.engine.Engine [generated in 0.00439s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2022-06-15 11:41:58,046 INFO sqlalchemy.engine.Engine ROLLBACK


In [239]:
scalar_subq = (
    select(address_table.c.email_address).
    where(address_table.c.user_id == user_table.c.id).
    order_by(address_table.c.id).
    limit(1).
    scalar_subquery()
)
update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


In [240]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(fullname="Pat")
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


In [242]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(
        {
            user_table.c.fullname: "Pat",
            address_table.c.email_address: "pat@aol.com",
        }
    )
)
from sqlalchemy.dialects import mysql
print(update_stmt.compile(dialect=mysql.dialect()))

UPDATE user_account, address SET address.email_address=%s, user_account.fullname=%s WHERE user_account.id = address.user_id AND address.email_address = %s


In [244]:
update_stmt = (
    update(some_table).
    ordered_values(
        (some_table.c.y, 20),
        (some_table.c.x, some_table.c.y + 10),
    )
)
print(update_stmt)

UPDATE some_table SET y=:y, x=(some_table.y + :y_1)


In [245]:
from sqlalchemy import delete
stmt = delete(user_table).where(user_table.c.name == "patrick")
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


In [246]:
delete_stmt = (
    delete(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com")
)
from sqlalchemy.dialects import mysql
print(delete_stmt.compile(dialect=mysql.dialect()))

DELETE FROM user_account USING user_account, address WHERE user_account.id = address.user_id AND address.email_address = %s


In [248]:
with engine.begin() as conn:
    result = conn.execute(
        update(user_table).
        values(fullname="Patrick McStar").
        where(user_table.c.name == "patrick")
    )
    print(result.rowcount)

2022-06-15 11:56:11,548 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 11:56:11,550 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2022-06-15 11:56:11,552 INFO sqlalchemy.engine.Engine [generated in 0.00171s] ('Patrick McStar', 'patrick')
1
2022-06-15 11:56:11,553 INFO sqlalchemy.engine.Engine COMMIT


In [249]:
update_stmt = (
    update(user_table).where(user_table.c.name == "patrick").
    values(fullname="Patrick the Star").
    returning(user_table.c.id, user_table.c.name)
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


In [250]:
delete_stmt = (
    delete(user_table).where(user_table.c.name == "patrick").
    returning(user_table.c.id, user_table.c.name)
)
print(delete_stmt)

DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


In [251]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

In [252]:
squidward

User('squidward', 'Squidward Tentacles')

In [257]:
session = Session(engine)
session.add(squidward)
session.add(krabs)

In [258]:
session.new

IdentitySet([User('squidward', 'Squidward Tentacles'), User('ehkrabs', 'Eugene H. Krabs')])

In [259]:
session.flush()

2022-06-15 12:12:01,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 12:12:01,451 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-06-15 12:12:01,452 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ('squidward', 'Squidward Tentacles')
2022-06-15 12:12:01,454 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-06-15 12:12:01,455 INFO sqlalchemy.engine.Engine [cached since 0.004196s ago] ('ehkrabs', 'Eugene H. Krabs')


In [260]:
squidward.id

4

In [261]:
krabs.id

5

In [262]:
some_squidward = session.get(User, 4)
some_squidward

User('squidward', 'Squidward Tentacles')

In [263]:
some_squidward is squidward

True

In [264]:
session.commit()

2022-06-15 12:14:01,192 INFO sqlalchemy.engine.Engine COMMIT


In [266]:
sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()

2022-06-15 12:15:02,678 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 12:15:02,679 INFO sqlalchemy.engine.Engine [cached since 2551s ago] ('sandy',)


In [267]:
sandy

User('sandy', 'Sandy Cheeks')

In [269]:
sandy.fullname = "Sandy Squirrel"

In [272]:
sandy in session.dirty

True

In [271]:
sandy

User('sandy', 'Sandy Squirrel')

In [274]:
sandy_fullname = session.execute(select(User.fullname).where(User.id==2)).scalar_one()

2022-06-15 12:17:54,521 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2022-06-15 12:17:54,523 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ('Sandy Squirrel', 2)
2022-06-15 12:17:54,525 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2022-06-15 12:17:54,526 INFO sqlalchemy.engine.Engine [generated in 0.00111s] (2,)


In [275]:
print(sandy_fullname)

Sandy Squirrel


In [276]:
sandy in session.dirty

False

In [277]:
session.new

IdentitySet([])

In [278]:
session.execute(
    update(User).
    where(User.name == "sandy").
    values(fullname="Sandy Squirrel Extraordinaire")
)

2022-06-15 12:19:54,906 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2022-06-15 12:19:54,908 INFO sqlalchemy.engine.Engine [generated in 0.00176s] ('Sandy Squirrel Extraordinaire', 'sandy')


In [279]:
sandy.fullname

'Sandy Squirrel Extraordinaire'

In [281]:
patrick = session.get(User, 3)

2022-06-15 12:20:45,754 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-06-15 12:20:45,755 INFO sqlalchemy.engine.Engine [generated in 0.00195s] (3,)


In [282]:
session.delete(patrick)

In [283]:
session.execute(select(User).where(User.name == "patrick")).first()

2022-06-15 12:21:21,426 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2022-06-15 12:21:21,428 INFO sqlalchemy.engine.Engine [generated in 0.00216s] (3,)
2022-06-15 12:21:21,431 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2022-06-15 12:21:21,432 INFO sqlalchemy.engine.Engine [generated in 0.00110s] (3,)
2022-06-15 12:21:21,435 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 12:21:21,436 INFO sqlalchemy.engine.Engine [cached since 2930s ago] ('patrick',)


In [284]:
patrick in session

False

In [285]:
squidward = session.get(User, 4)

In [286]:
session.execute(delete(User).where(User.name == "squidward"))

2022-06-15 12:22:46,784 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.name = ?
2022-06-15 12:22:46,786 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ('squidward',)


In [287]:
squidward in session

False

In [288]:
session.rollback()

2022-06-15 12:23:23,056 INFO sqlalchemy.engine.Engine ROLLBACK


In [289]:
sandy.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f1c2a43a350>}

In [290]:
sandy.fullname

2022-06-15 12:23:45,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 12:23:45,801 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-06-15 12:23:45,802 INFO sqlalchemy.engine.Engine [cached since 562.1s ago] (2,)


'Sandy Cheeks'

In [291]:
sandy.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f1c2a43a350>,
 'id': 2,
 'fullname': 'Sandy Cheeks',
 'name': 'sandy'}

In [292]:
patrick in session

True

In [293]:
session.execute(select(User).where(User.name == "patrick")).scalar_one is patrick

2022-06-15 12:24:46,225 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-06-15 12:24:46,227 INFO sqlalchemy.engine.Engine [cached since 3135s ago] ('patrick',)


False

In [294]:
session.close()

2022-06-15 12:25:03,349 INFO sqlalchemy.engine.Engine ROLLBACK


In [295]:
squidward.name

DetachedInstanceError: Instance <User at 0x7f1c2a46efe0> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/14/bhk3)

In [296]:
session.add(squidward)
squidward.name

2022-06-15 12:25:58,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-15 12:25:58,684 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-06-15 12:25:58,685 INFO sqlalchemy.engine.Engine [cached since 695s ago] (4,)


'squidward'